# 💤 Drowsiness Detection with YOLOv8 (Local)

This notebook performs real-time drowsiness detection using a **custom-trained YOLOv8 model**.  
The system monitors facial features (like eyes and face) from a webcam feed and alerts the user if eyes remain closed for more than 2 seconds.

---

## 📦 Step 1: Install Dependencies

Before running the detection code, make sure the following libraries are installed:

- `ultralytics`
- `torch`
- `opencv-python`
- `pyttsx3`

🛠️ Uncomment the line below and run it **once** to install dependencies:

In [ ]:
# !pip install -r requirements.txt

---

## ⚙️ (Optional) Install GPU Support (NVIDIA Only)

If you're using an **NVIDIA GPU** and want to test model inference with hardware acceleration,  
you can install the CUDA-enabled version of PyTorch.

> ⚠️ This step is **not recommended for basic testing or image-level demo**.  
> The download may exceed **2.5 GB**, and the CPU version already works fine for testing purposes.

🛠️ Only run this if you're setting up for real-time or large-scale processing and have an NVIDIA GPU.

Uncomment the line below and run:

In [ ]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

---

## 🖼️ Image-Based Drowsiness Detection
This section runs YOLOv8 detection on a static image and displays the annotated result with class labels.

📌 Useful for verifying the model on single inputs before running real-time detection.

In [ ]:
import cv2
import torch
from ultralytics import YOLO

# Load the custom-trained YOLOv8 model
model = YOLO('Source\Models\model.pt')

# Path to the input image for prediction
img = 'Source\Images\input.jpg'

# Perform prediction on the input image
results = model.predict(
    source=img,
    save=False,   # Don't save the result to file
    show=False,   # Don't display using built-in viewer
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
)

# Plot the detection results (draw bounding boxes, labels, etc.)
result_img = results[0].plot()

# Create a resizable OpenCV window
cv2.namedWindow("Prediction", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Prediction", 800, 600)  # Set custom window size

# Display the annotated result image
cv2.imshow("Prediction", result_img)

# Wait for user to press 'q', 'Q', or ESC to exit
while True:
    key = cv2.waitKey(1)
    if key in [ord("q"), ord("Q"), 27]:
        break

# Clean up and close all OpenCV windows
cv2.destroyAllWindows()


---

## 🎥 Step 3: Real-Time Drowsiness Detection (Webcam)
This section uses your webcam to capture frames in real time and monitor eye activity.

- If eyes remain closed for more than 2 seconds, the system issues a voice alert.

- Detection stops when eyes are open again.

- Press ESC to exit the detection window.

In [ ]:
import cv2
import pyttsx3
import time as t
import torch
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('Source\Models\model.pt')

# Start capturing video from the default webcam (0)
cap = cv2.VideoCapture(0)

# Counter to track eye-closed duration
count = 0

# Create a resizable OpenCV window
cv2.namedWindow("Drowsiness Detection", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Drowsiness Detection", 800, 600)  # Optional window size adjustment

while True:
    success, frame = cap.read()  # Capture frame from webcam
    frame = cv2.flip(frame, 1)   # Flip the frame horizontally (mirror view)

    # Run YOLOv8 detection on the current frame
    results = model.predict(
        source=frame,
        show=False,
        save=False,
        classes=[0, 1, 2],  # Detect only specific classes: face, open eyes, closed eyes
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    )

    # Get list of detected class names
    names = model.names
    check = []
    for r in results:
        for c in r.boxes.cls:
            check.append(names[int(c)])

    # Logic to detect drowsiness based on eye state
    if "face" in check:
        if "close eyes" in check:
            if count == 0:
                start_time = float(t.time())  # Start timing when eyes first close
                count = 1
            end_time = float(t.time())
            run_time = end_time - start_time

            # Trigger alert if eyes remain closed for more than 2 seconds
            if run_time > 2:
                print("alert")
                alert = pyttsx3.init()
                alert.say("Wake up")
                alert.runAndWait()
                alert = None
                print(run_time)
        elif "open eyes" in check:
            # Reset alarm and timer when eyes are open
            alert = None
            print("no alarm")
            count = 0

    # Annotate frame with detection results
    annotated_frame = results[0].plot()

    # Show annotated frame in the OpenCV window
    cv2.imshow("Drowsiness Detection", annotated_frame)

    # Exit loop when ESC key is pressed
    if cv2.waitKey(2) & 0xFF == 27:
        break

# Release webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


<>:8: SyntaxWarning: invalid escape sequence '\M'
<>:8: SyntaxWarning: invalid escape sequence '\M'
C:\Users\WORK\AppData\Local\Temp\ipykernel_2692\3142275224.py:8: SyntaxWarning: invalid escape sequence '\M'
  model = YOLO('Source\Models\model.pt')



0: 480x640 (no detections), 332.8ms
Speed: 2.7ms preprocess, 332.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 1 open eyes, 301.8ms
Speed: 1.8ms preprocess, 301.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
no alarm

0: 480x640 1 face, 1 open eyes, 306.0ms
Speed: 1.7ms preprocess, 306.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
no alarm

0: 480x640 1 face, 1 open eyes, 285.9ms
Speed: 1.6ms preprocess, 285.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
no alarm

0: 480x640 1 face, 1 open eyes, 269.2ms
Speed: 1.5ms preprocess, 269.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
no alarm

0: 480x640 1 face, 1 open eyes, 307.6ms
Speed: 1.5ms preprocess, 307.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
no alarm

0: 480x640 1 face, 1 open eyes, 427.2ms
Speed: 2.2ms preprocess, 427.2ms inference, 1.5ms postprocess per image at shape (1,